# GeoTrellis Operations

In [8]:
from pyrasterframes import *
from pyrasterframes.rasterfunctions import *
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder. \
    master("local[*]"). \
    appName("RasterFrames"). \
    config("spark.ui.enabled", "false"). \
    getOrCreate(). \
    withRasterFrames()
    
rf = spark.read.geotiff("../samples/L8-B8-Robinson-IL.tiff")

GeoTrellis provides a [rich set of Map Algebra operations](https://docs.geotrellis.io/en/latest/guide/core-concepts.html#map-algebra) and other tile processing features that can be used with RasterFrames via Spark's UDF support.

Here's an example creating a UDFs to invoke the `equalize` transformation on each tile in a RasterFrame, and then compute the resulting per-tile mean of it.

In [ ]:
# Change to not use UDFs any more
equalizer = udf((t: Tile) => t.equalize())
equalized = rf.select(equalizer($"tile") as "equalized")
equalized.select(tileMean($"equalized") as "equalizedMean").show(5, false)

Here's an example downsampling a tile and rendering each tile as a matrix of numerical values.

In [ ]:
downsample = udf((t: Tile) => t.resample(4, 4))
downsampled = rf.select(renderAscii(downsample($"tile")) as "minime")
downsampled.show(5, false)